In [1]:
# !pip install --upgrade transformers

In [2]:
# !pip install simpletransformers

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
#importing required libraries

import matplotlib.pyplot as plt
import seaborn as sns
 
import os
import gc
import re
from tqdm.autonotebook import tqdm
 
from sklearn.metrics import accuracy_score,f1_score


import sklearn

import torch
from simpletransformers.classification import ClassificationModel

 
from wordcloud import STOPWORDS
from wordcloud import WordCloud
import numpy as np 
import pandas as pd


# from imblearn.over_sampling import SMOTE,SMOTENC
# oversample = SMOTE()

from transformers import AutoTokenizer
model_name ="bert-base-uncased"



/tmp/ipykernel_4483/1074144912.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
2022-12-03 07:04:07.427773: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-03 07:04:07.611199: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-03 07:04:08.266703: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dyna

In [3]:
train = pd.read_csv("bert_preprocessed_train.csv")
test = pd.read_csv("bert_preprocessed_test.csv")

In [4]:
train.head(2)

,tweet,label
0,keyshawnswag lmfao this cat started beating th...,1
1,rt digaveliavelife lol i be eatin da shit outt...,1


In [5]:


tokenizer = AutoTokenizer.from_pretrained(model_name)
train['tweet'] = train['tweet'].astype(str)
test['tweet'] = test['tweet'].astype(str)
train_df,val_df = train_test_split(train,test_size =0.15, random_state =500)
val_df.head()

train_encodings = tokenizer(list(train_df['tweet']), truncation=True, padding=True)
val_encodings = tokenizer(list(val_df['tweet']), truncation=True, padding=True)
train_full_encodings = tokenizer(list(train['tweet']), truncation=True, padding=True)
# test_encodings = tokenizer(test_texts, truncation=True, padding=True)
import torch

class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels,trainset=False):

        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

            
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

    


train_dataset = HateSpeechDataset(train_encodings, list(train_df['label']),trainset=False)
val_dataset = HateSpeechDataset(val_encodings, list(val_df['label']))
train_full_dataset = HateSpeechDataset(train_full_encodings, list(val_df['label']))
# test_dataset = HateSpeechDataset(test_encodings, test_labels)

In [6]:
class_weights = (1- (train['label'].value_counts().sort_index()/len(train))).values

In [7]:
class_weights

array([0.94265106, 0.22334308, 0.83400585])

In [8]:

class_weights = torch.from_numpy(class_weights).float().to('cuda')

In [9]:
train.rename(columns={"label":"labels"})

,tweet,labels
0,keyshawnswag lmfao this cat started beating th...,1
1,rt digaveliavelife lol i be eatin da shit outt...,1
2,rt queenreenie how bitch how 8220vontethekidd ...,1
3,floppy bird chalmers over here taking after hi...,2
4,boolin in the bando wit a few bitches,1
...,...,...
19821,good morning to lightskin dyke bitches with st...,0
19822,making brownies,2
19823,you all think pauladeen would call my daughter...,1
19824,too little too late bitch ass nigga,1


In [10]:
from transformers import Trainer
class weightedLossTrainer(Trainer):
    def compute_loss(self,model,inputs,return_outputs=False):
        outputs = model(**inputs)
        logits = outputs.get("logits")
        labels = inputs['labels']
        loss_func = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss=loss_func(logits,labels)
        return (loss,outputs) if return_outputs else loss

In [11]:
from transformers import AutoModelForSequenceClassification,BertConfig
# config = BertConfig.from_pretrained(model_name)   
# model = AutoModelForSequenceClassification.from_config(config)  
model = AutoModelForSequenceClassification.from_pretrained(model_name,num_labels=3)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
from sklearn.metrics import f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels,preds,average="weighted")
    return {"f1":f1}
    

In [13]:
from transformers import TrainingArguments
batch_size = 32
logging_steps = len(train)//batch_size
output_dir = "results"
trainArgs = TrainingArguments(output_dir=output_dir,
                             num_train_epochs=2,
                             learning_rate=1e-5,
                             per_device_train_batch_size=batch_size,
                             per_device_eval_batch_size=batch_size,
                             weight_decay=0.01,
                             evaluation_strategy="epoch",
                             logging_steps =logging_steps,
                            
                             push_to_hub=False)

In [14]:
trainer = weightedLossTrainer(model=model,
                             args=trainArgs,
                             compute_metrics=compute_metrics,
                             train_dataset=train_dataset,
                            eval_dataset = val_dataset,
                             tokenizer=tokenizer)

In [15]:
trainer.train()
print(model_name)

/home/dcll4/miniconda3/envs/nlp/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16852
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1054
  Number of trainable parameters = 109484547
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: ramyanee. Use `wandb login --relogin` to force relogin


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1
1,No log,0.381356,0.896630
2,0.504500,0.371510,0.900929


Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-500/tokenizer_config.json
Special tokens file saved in results/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2974
  Batch size = 32
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in results/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 2974
  Batch size = 32


Training completed. Do not forget to share your model on huggingface.co/models =)




bert-base-uncased


In [16]:
test_encodings = [tokenizer(x, truncation=True, padding=True) for x in list(test['tweet'])]
# item = {key: torch.tensor(val[idx]) for key, val in test_encodings.items()}


In [17]:
predictions = trainer.predict(test_encodings)


# print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 4957
  Batch size = 32


In [18]:
preds = np.argmax(predictions.predictions, axis=-1)

In [19]:
submission = pd.DataFrame(zip(preds,list(test['id'])),columns=['label','id'])

In [20]:
submission.to_csv('bert_base_uncased_2ep_preprocessed.csv',index=None)

In [21]:
submission

,label,id
0,0,0
1,0,1
2,1,2
3,1,3
4,1,4
...,...,...
4952,0,4952
4953,2,4953
4954,2,4954
4955,2,4955
